# CSC2042S Assignment 1: Data Preprocessing

This notebook implements Part 1 (Data Preprocessing) of the assignment. It expects the WDI dataset to be located in `CSC2042S-Assignment1-Data/WDICSV.csv`.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

DATA_DIR = Path('CSC2042S-Assignment1-Data')
WDI_PATH = DATA_DIR / 'WDICSV.csv'

In [ ]:
def load_wdi_dataset(path: Path) -> pd.DataFrame:
    """Load WDI CSV into tidy format with rows as country-year and columns as indicators."""
    raw = pd.read_csv(path)
    id_vars = ['Country Name','Country Code','Indicator Name','Indicator Code']
    year_cols = [c for c in raw.columns if c.isdigit()]
    tidy = raw.melt(id_vars=id_vars, value_vars=year_cols,
                    var_name='Year', value_name='Value').dropna(subset=['Value'])
    tidy['Year'] = tidy['Year'].astype(int)
    pivot = tidy.pivot_table(index=['Country Name','Country Code','Year'],
                             columns='Indicator Code', values='Value').reset_index()
    return pivot

In [ ]:
def preprocess(df: pd.DataFrame, feature_thresh: float=0.3, sample_thresh: float=0.7) -> pd.DataFrame:
    feature_coverage = df.notna().mean()
    df = df.loc[:, feature_coverage >= feature_thresh]
    sample_coverage = df.notna().mean(axis=1)
    df = df.loc[sample_coverage >= sample_thresh]
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())
    return df

In [ ]:
def normalize(df: pd.DataFrame) -> pd.DataFrame:
    scaler = MinMaxScaler()
    numeric = df.select_dtypes(include=[np.number])
    scaled = scaler.fit_transform(numeric)
    df[numeric.columns] = scaled
    return df

In [ ]:
def visualize_tsne(df: pd.DataFrame, perplexity: float=30.0, random_state: int=0):
    tsne = TSNE(n_components=2, perplexity=perplexity, random_state=random_state)
    emb = tsne.fit_transform(df.select_dtypes(include=[np.number]))
    plt.figure(figsize=(6,5))
    plt.scatter(emb[:,0], emb[:,1], s=5)
    plt.title('t-SNE Visualization')
    plt.xlabel('Dim 1')
    plt.ylabel('Dim 2')
    plt.show()

In [ ]:
# Load dataset
data = load_wdi_dataset(WDI_PATH)
print(f'Loaded dataset with shape {data.shape}')
data.head()

In [ ]:
# Preprocess dataset
processed = preprocess(data, feature_thresh=0.3, sample_thresh=0.7)
print(f'After preprocessing: {processed.shape}')
processed.head()

In [ ]:
# Normalize features
normalized = normalize(processed)
normalized.head()

In [ ]:
# Visualize with t-SNE
visualize_tsne(normalized.drop(columns=['Country Name','Country Code','Year']))